In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 17.6 MB/s eta 0:00:00


In [2]:
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

In [3]:
def hydrogen_hamiltonian(coordinates):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    return qml.qchem.molecular_hamiltonian(["H", "Li"], coordinates, charge=0, basis="STO-3G",active_electrons=2)[0]


def hf(electrons, num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.

    Args:
        electrons (int): The number of electrons in the hydrogen molecule.
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
    # Put your solution here #
    return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

In [4]:
def num_electrons(charge):
    """The total number of electrons in the hydrogen molecule.

    Args:
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (int): The number of electrons.
    """
    return 2-charge

In [5]:
def depth(qnode):
    def _fn(*args, **kwargs):
        qnode.construct(args, kwargs)
        return qnode.qtape.get_depth()
    return _fn

Using exact diagonalization to get the ground state and first excited state.

In [7]:
L=1.5
coordinates = np.array([0.0, 0.0, -L/2, 0.0, 0.0, L/2]) #np.array([-L/2/np.sqrt(3), L/2, 0.0, L/np.sqrt(3), 0.0, 0.0,-L/2/np.sqrt(3), -L/2, 0.0])
charge=0
hamiltonian = hydrogen_hamiltonian(np.array(coordinates))
H_matrix=qml.matrix(hamiltonian)

In [8]:
vals, vecs = np.linalg.eigh(H_matrix)

inds=np.argsort(vals)
eng=vals[inds]
vec=vecs[:,inds]

print('The ground state energy:',eng[0])


The ground state energy: -7.626624465547493


In [9]:
def vec_check(vec,electron_num):
    for i,e in enumerate(vec):
        if np.abs(e) > 1e-8:
            e_occ=bin(i)[2:]
            lst=' '.join(e_occ).split(' ')
            cv=tuple(map(int, lst))
            e_num=sum(cv)
            if e_num!=electron_num:
                return False
    return True

In [10]:
sz=qml.qchem.spinz(len(hamiltonian.wires))
SZ=qml.matrix(sz)
Stop=[False,False,False]
elec_num=2
for i in range(1,len(eng)): # Finding first excited state with -1 Sz
    Sz=vec[:,i].dot(SZ.dot(np.transpose(np.conjugate(vec[:,i]))))
    #print(Sz)

    if np.abs(Sz) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=1:',eng[i])
        Stop[0]=True

    elif np.abs(Sz-1) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=0:',eng[i])
        Stop[1]=True

    elif np.abs(Sz+1) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=-1:',eng[i])
        Stop[2]=True

    if np.all(Stop):
        break


The first excied state energy with sz=0: -7.522914627103037
The first excied state energy with sz=1: -7.522914627103034
The first excied state energy with sz=-1: -7.52291462710303


In [50]:
def bit_to_vec(bitstring_dict,num_qubits):
    vec=np.zeros(2**num_qubits)
    amp=0
    for i in range(2**num_qubits):
        bitstr=bin(i)[2:]
        if len(bitstr) < num_qubits:
            bitstr=(num_qubits-len(bitstr))*'0'+bitstr

        if bitstr in bitstring_dict:
            vec[i]=bitstring_dict[bitstr]
            amp+=np.abs(bitstring_dict[bitstr])**2
    vec=vec/np.sqrt(amp)
    return vec

In [90]:
class VQD_ch():
    def __init__(self,state_lst,num_electrons,symbol,geometry,charge) -> None:
        self.num_electrons=num_electrons
        self.symbol=symbol
        self.geometry=geometry
        self.charge=charge

        self.hamiltonian=self.Ch_hamiltonian(symbol,geometry,num_electrons,charge)
        self.num_qubits = len(self.hamiltonian.wires)
        self.sz=qml.qchem.spinz(self.num_qubits)
        self.singles, self.doubles = qml.qchem.excitations(self.num_electrons, self.num_qubits)
        self.state_lst=[]
        for stat in state_lst:
            self.state_lst.append(bit_to_vec(stat,self.num_qubits))
            #bit_lst=' '.join(stat).split(' ')
            #self.state_lst.append(list(map(int, bit_lst)))
        #self.stat_bitstr=state_lst

        self.weight_dict={}
        self.th_state=0
        self.Eng_lst=[]

        self.dev = qml.device("default.qubit", wires=self.num_qubits)
        self.cost_h_fn = qml.QNode(self.circuit, self.dev)
        self.Sz_fn = qml.QNode(self.circuit_sz, self.dev)
        self.cost_fid_fn = qml.QNode(self.circuit_fid, self.dev)
        self.num_ly_lst=[]

    def Ch_hamiltonian(self,symbol,geometry,num_electrons,charge):
        return qml.qchem.molecular_hamiltonian(symbol, geometry, charge=charge,\
                                               basis="STO-3G",active_electrons=num_electrons)[0]

    def hf(self,electrons, num_qubits):
        return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

    def circuit(self,state,weights):
        #qml.BasisState(state, wires=range(self.num_qubits))
        qml.StatePrep(state, wires=range(self.num_qubits))
        d_l=len(self.doubles)
        s_l=len(self.singles)
        for j in range(self.num_layer):
            for i in range(len(self.singles)):
                qml.SingleExcitation(weights[i+j*(d_l+s_l)], wires=self.singles[i])
            for i in range(len(self.doubles)):
                qml.DoubleExcitation(weights[i+s_l+j*(d_l+s_l)], wires=self.doubles[i])
        return qml.expval(self.hamiltonian)


    def circuit_sz(self,state,weights):
        #qml.BasisState(state, wires=range(self.num_qubits))
        qml.StatePrep(state, wires=range(self.num_qubits))
        d_l=len(self.doubles)
        s_l=len(self.singles)
        for j in range(self.num_layer):
            for i in range(len(self.singles)):
                qml.SingleExcitation(weights[i+j*(d_l+s_l)], wires=self.singles[i])
            for i in range(len(self.doubles)):
                qml.DoubleExcitation(weights[i+s_l+j*(d_l+s_l)], wires=self.doubles[i])
        return qml.expval(self.sz)

    def circuit_fid(self,state,params,weights,ind):
        #qml.BasisState(state, wires=range(num_qubits))
        qml.StatePrep(state, wires=range(self.num_qubits))
        Re_w=weights.copy()[::-1]
        d_l=len(self.doubles)
        s_l=len(self.singles)
        for j in range(self.num_layer):
            for i in range(len(self.singles)):
                qml.SingleExcitation(params[i+j*(d_l+s_l)], wires=self.singles[i])
            for i in range(len(self.doubles)):
                qml.DoubleExcitation(params[i+s_l+j*(d_l+s_l)], wires=self.doubles[i])

        Re_single=self.singles.copy()[::-1]
        Re_double=self.doubles.copy()[::-1]
        for j in range(self.num_ly_lst[ind]):
            for i in range(len(self.doubles)):
                qml.DoubleExcitation(-Re_w[i+j*(d_l+s_l)], wires=Re_double[i])
            for i in range(len(self.singles)):
                qml.SingleExcitation(-Re_w[i+d_l+j*(d_l+s_l)], wires=Re_single[i])

        return qml.state() #qml.probs(wires=range(num_qubits))

    def cost_fn(self,params):

        cost=self.cost_h_fn(self.stat,params)
        for ind in range(self.th_state):
            #print('hi')
            #stat_ind=int(self.stat_bitstr[ind],base=2)
            coj_stat=np.conjugate(np.transpose(self.state_lst[ind]))
            final_stat=self.cost_fid_fn(self.stat,params,self.weight_dict[ind],ind)
            #fid=self.cost_fid_fn(self.stat,params,self.weight_dict[ind],ind)
            fid=np.abs(np.dot(coj_stat,final_stat))**2
            cost+=self.beta*fid#[stat_ind]
        return cost

    def run(self,beta=3.0,epochs=400,stepsize=0.5,num_layer=2,cov=1e-6,print_train=True):

        if self.th_state==len(self.state_lst):
            print('Run of states')
            return None
        self.beta=beta
        self.stat=self.state_lst[self.th_state]
        self.num_layer=num_layer
        opt = qml.AdamOptimizer(stepsize=stepsize)
        self.weights =np.zeros(num_layer*len(self.singles+self.doubles), requires_grad=True)

        i=0
        iter=[]
        cost_val=[]
        Lowest_cost=0

        cov=cov
        E_pev=0

        for _ in range(epochs):
            self.weights = opt.step(self.cost_fn, self.weights)
            iter.append(i)
            cost=self.cost_fn(self.weights)

            E0=self.cost_h_fn(self.stat,self.weights)

            cost_val.append(E0)

            i=i+1
            if print_train:
                print('iter:',i)
                print(' E0:',E0,' Sz:',self.Sz_fn(self.stat,self.weights))
                print(cost)
            if np.abs(E0-E_pev)<cov:
                break

            E_pev=E0
        self.Eng_lst=E0
        self.weight_dict[self.th_state]=self.weights
        self.th_state+=1
        self.num_ly_lst.append(num_layer)
        return self.cost_h_fn(self.stat,self.weights)

In [91]:
L=1.5

#a0=0.529177210903
#L=r/a0
symbol=["H", "Li"]
geometry=np.array([0.0, 0.0, -L/2, 0.0, 0.0, L/2])

charge=0
num_electrons=2

g_stat={'1100000000': 1}
e_stat_0={'1001000000': 1}
e_stat_01={'1001000000': 1}
stat_lst=[g_stat,e_stat_0,e_stat_01]
VQD=VQD_ch(stat_lst,num_electrons,symbol,geometry,charge)

E0=VQD.run(num_layer=1)

iter: 1
 E0: -7.475187554557702  Sz: 0.0
-7.475187554557702
iter: 2
 E0: -7.53914631149875  Sz: -2.7755575615628914e-17
-7.53914631149875
iter: 3
 E0: -7.58517297151212  Sz: 2.7755575615628914e-17
-7.58517297151212
iter: 4
 E0: -7.53149154544308  Sz: 5.551115123125783e-17
-7.53149154544308
iter: 5
 E0: -7.530700124349723  Sz: 0.0
-7.530700124349723
iter: 6
 E0: -7.56655319979495  Sz: 2.7755575615628914e-17
-7.56655319979495
iter: 7
 E0: -7.590039956292802  Sz: 0.0
-7.590039956292802
iter: 8
 E0: -7.58705149685098  Sz: -8.326672684688674e-17
-7.58705149685098
iter: 9
 E0: -7.580778730576064  Sz: 0.0
-7.580778730576064
iter: 10
 E0: -7.585438382869366  Sz: -2.7755575615628914e-17
-7.585438382869366
iter: 11
 E0: -7.599889175920599  Sz: -2.7755575615628914e-17
-7.599889175920599
iter: 12
 E0: -7.6126141810066414  Sz: -2.7755575615628914e-17
-7.6126141810066414
iter: 13
 E0: -7.608241253904193  Sz: 2.7755575615628914e-17
-7.608241253904193
iter: 14
 E0: -7.597549385444611  Sz: 2.7755575615

In [92]:
E1=VQD.run(num_layer=2)

/usr/local/lib/python3.10/dist-packages/autograd/numpy/numpy_wrapper.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)


iter: 1
 E0: -7.3972499747055265  Sz: 0.0
-7.102826339199836
iter: 2
 E0: -7.1736326245868405  Sz: -5.551115123125783e-17
-6.8669522157772835
iter: 3
 E0: -7.192036034926721  Sz: 5.551115123125783e-17
-7.10544296331033
iter: 4
 E0: -7.337519708748939  Sz: -2.7755575615628914e-17
-7.329740530669701
iter: 5
 E0: -7.475135755972643  Sz: -8.326672684688674e-17
-7.434768870645787
iter: 6
 E0: -7.443184698045552  Sz: 0.0
-7.437687521510465
iter: 7
 E0: -7.353707197026117  Sz: -5.551115123125783e-17
-7.35101327519098
iter: 8
 E0: -7.323759531836687  Sz: 0.0
-7.31320116355041
iter: 9
 E0: -7.359933718063774  Sz: 0.0
-7.348121925110172
iter: 10
 E0: -7.437233178310966  Sz: -2.7755575615628914e-17
-7.427548109296166
iter: 11
 E0: -7.502336711223109  Sz: -5.551115123125783e-17
-7.494438301695373
iter: 12
 E0: -7.504088753153259  Sz: -1.1102230246251565e-16
-7.496803545376954
iter: 13
 E0: -7.4612236582260145  Sz: 2.7755575615628914e-17
-7.454750494403643
iter: 14
 E0: -7.430438007776041  Sz: -8.3

In [93]:
E2=VQD.run(beta=4.0,num_layer=3)

iter: 1
 E0: -7.148627902725036  Sz: 0.0
-5.566979962117046
iter: 2
 E0: -7.074667070201315  Sz: 0.0
-6.232247992652986
iter: 3
 E0: -7.07163065993168  Sz: 0.0
-6.403987281345381
iter: 4
 E0: -7.015246669804305  Sz: 2.7755575615628914e-17
-6.934996934872965
iter: 5
 E0: -7.133219258883945  Sz: 0.0
-6.82647490190445
iter: 6
 E0: -7.207550553591435  Sz: 2.7755575615628914e-17
-6.972744096461448
iter: 7
 E0: -7.097431682272925  Sz: 8.326672684688674e-17
-7.089549685221541
iter: 8
 E0: -7.038834917844877  Sz: 1.3877787807814457e-17
-6.874155450047245
iter: 9
 E0: -7.026351703488842  Sz: -1.3877787807814457e-17
-6.924267431801992
iter: 10
 E0: -7.066049504229553  Sz: 2.7755575615628914e-17
-7.040990908281397
iter: 11
 E0: -7.147902201984084  Sz: 2.7755575615628914e-17
-7.080009370118756
iter: 12
 E0: -7.1979032483438425  Sz: -2.7755575615628914e-17
-7.178136709173973
iter: 13
 E0: -7.217457539147862  Sz: 0.0
-7.194125787412354
iter: 14
 E0: -7.228560236827409  Sz: 0.0
-7.15185236506964
iter

In [39]:
electrons = 2
num_qubits = len(hamiltonian.wires)

S2 = qml.qchem.spin2(electrons, num_qubits)
sz=qml.qchem.spinz(num_qubits)

singles, doubles = qml.qchem.excitations(electrons, num_qubits)
pool_operators=singles+doubles

num_qubits = len(hamiltonian.wires)
hf_state = hf(electrons, num_qubits)

dev = qml.device("default.qubit", wires=num_qubits)
print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")

bitstr=''.join(map(str, hf_state))

g_trial_stat={bitstr: 1}
e_trial_stat_szp1={'1010000000': 1}
e_trial_stat_szm1={'0101000000': 1}
e_trial_stat_sz0={'1001000000': 1,'0110000000': 1}

def bit_to_vec(bitstring_dict,num_qubits):
    vec=np.zeros(2**num_qubits)
    amp=0
    for i in range(2**num_qubits):
        bitstr=bin(i)[2:]
        if len(bitstr) < num_qubits:
            bitstr=(num_qubits-len(bitstr))*'0'+bitstr

        if bitstr in bitstring_dict:
            vec[i]=bitstring_dict[bitstr]
            amp+=np.abs(bitstring_dict[bitstr])**2
    vec=vec/np.sqrt(amp)
    return vec

g_stat=bit_to_vec(g_trial_stat,num_qubits)
#e_stat_szp1=bit_to_vec(e_trial_stat_szp1,num_qubits)

def circuit(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    return qml.expval(hamiltonian)

def circuit_sz(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    return qml.expval(sz)


cost_h_fn = qml.QNode(circuit, dev)
Sz_fn = qml.QNode(circuit_sz, dev)

def cost_fn(params):
    return cost_h_fn(g_stat,params)

weights =np.zeros(len(doubles + singles), requires_grad=True)

opt = qml.AdamOptimizer(stepsize=0.5)

i=0
iter=[]
cost_val=[]
Lowest_E=0
best_weights=0

cov=1e-6
E0_pev=0


for _ in range(400):
    weights = opt.step(cost_fn, weights)
    iter.append(i)

    E0=cost_h_fn(g_stat,weights)

    if Lowest_E>E0:
        Lowest_E=E0
        best_weights=weights
    cost_val.append(E0)

    i=i+1
    print('Need iter:',i,' E0:',E0,' Sz:',Sz_fn(g_stat,weights))
    if np.abs(E0-E0_pev)<cov:
        break

    E0_pev=E0


The original vqe use  224  cnot gates
The original vqe use  624  gates
Need iter: 1  E0: -7.475187554557702  Sz: 0.0
Need iter: 2  E0: -7.53914631149875  Sz: -2.7755575615628914e-17
Need iter: 3  E0: -7.58517297151212  Sz: 2.7755575615628914e-17
Need iter: 4  E0: -7.53149154544308  Sz: 5.551115123125783e-17
Need iter: 5  E0: -7.530700124349723  Sz: 0.0
Need iter: 6  E0: -7.56655319979495  Sz: 2.7755575615628914e-17
Need iter: 7  E0: -7.590039956292802  Sz: 0.0
Need iter: 8  E0: -7.58705149685098  Sz: -8.326672684688674e-17
Need iter: 9  E0: -7.580778730576064  Sz: 0.0
Need iter: 10  E0: -7.585438382869366  Sz: -2.7755575615628914e-17
Need iter: 11  E0: -7.599889175920599  Sz: -2.7755575615628914e-17
Need iter: 12  E0: -7.6126141810066414  Sz: -2.7755575615628914e-17
Need iter: 13  E0: -7.608241253904193  Sz: 2.7755575615628914e-17
Need iter: 14  E0: -7.597549385444611  Sz: 2.7755575615628914e-17
Need iter: 15  E0: -7.5982604705574746  Sz: 0.0
Need iter: 16  E0: -7.609303957356188  Sz: 

In [ ]:
def circuit_h(state,params):
    Re_w=weights[::-1]
    qml.StatePrep(state, wires=range(num_qubits))

    for i in range(len(singles)):
        qml.SingleExcitation(params[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(params[i], wires=doubles[i])

    return qml.expval(hamiltonian)

def circuit_fid(state,params,weights):
    Re_w=weights[::-1]
    qml.StatePrep(state, wires=range(num_qubits))

    for i in range(len(singles)):
        qml.SingleExcitation(params[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(params[i], wires=doubles[i])

    for i in range(len(singles)):
        qml.SingleExcitation(params[i+2*len(doubles)+len(singles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(params[i+len(doubles)+len(singles)], wires=doubles[i])

    for i, double in enumerate(doubles[::-1]):
        qml.DoubleExcitation(-Re_w[i], wires=double)
    for i, single in enumerate(singles[::-1]):
        qml.SingleExcitation(-Re_w[i+len(doubles)], wires=single)
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    return qml.probs(wires=range(len(hamiltonian.wires)))

cost_h_fn = qml.QNode(circuit_h, dev)
cost_fid_fn = qml.QNode(circuit_fid, dev)
Sz_fn = qml.QNode(circuit_sz, dev)

params =np.zeros(2*len(doubles + singles), requires_grad=True)

e_trial_stat_sz0={'1010000000': 1}
e_stat_sz0=bit_to_vec(e_trial_stat_sz0,num_qubits)

def cost_fn(params):
    return cost_h_fn(e_stat_sz0,params)+cost_fid_fn(e_stat_sz0,params,weights)[0]

opt = qml.AdamOptimizer(stepsize=0.5)


i=0
iter=[]
cost_val=[]
Lowest_E=0
best_weights=0

cov=1e-6
E0_pev=0


for _ in range(400):
    params = opt.step(cost_fn, params)
    iter.append(i)

    E0=cost_h_fn(e_stat_sz0,params)

    if Lowest_E>E0:
        Lowest_E=E0
        best_weights=params
    cost_val.append(E0)

    i=i+1
    print('Need iter:',i,' E0:',E0,' Sz:',Sz_fn(e_stat_sz0,params))
    print("Cost function",cost_fn(params))
    if np.abs(E0-E0_pev)<cov:
        break

    E0_pev=E0

In [ ]:

electrons = 2
num_qubits = len(hamiltonian.wires)

singles, doubles = qml.qchem.excitations(electrons, num_qubits)
pool_operators=singles+doubles

num_qubits = len(hamiltonian.wires)
hf_state = hf(electrons, num_qubits)

def circuit_1(state,weights,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(weights[i], wires=excitation)
        else:
            qml.SingleExcitation(weights[i], wires=excitation)
    return qml.expval(hamiltonian)

def circuit_sz(state,weights,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(weights[i], wires=excitation)
        else:
            qml.SingleExcitation(weights[i], wires=excitation)
    return qml.expval(sz)

dev = qml.device("default.qubit", wires=num_qubits)#

cost_h_fn = qml.QNode(circuit_1, dev, expansion_strategy="device")
Sz_fn = qml.QNode(circuit_sz, dev)

def cost_fn(params, excitations):
    return cost_h_fn(g_stat,params, excitations)


epochs=50

energy=[]
operator_circuits=[]# doubles_select+singles_select

weights = [] #[0.0] * len(operator_circuits)
weights=np.array(weights)

opt = opt = qml.GradientDescentOptimizer(stepsize=0.25)

circuit_gradient = qml.grad(cost_fn, argnum=0)


E0 = cost_h_fn(g_stat,weights, operator_circuits)

print(f"Epoch = 0, G Energy = {E0:.8f} Ha")

print("Number of gates = {}\n".format(len(operator_circuits)))

configs=qml.specs(cost_h_fn)(g_stat,weights, excitations=operator_circuits)

#print(configs)
print("Circuit depth {}\n".format(configs['resources'].depth))

conv=1e-7
E0_pev=0
E1_pev=0
E2_pev=0
E3_pev=0
for n in range(epochs):

    Test_set=operator_circuits+pool_operators
    Test_params=list(weights)+[0.0]*len(pool_operators)
    Test_params=np.array(Test_params)

    test_grads = circuit_gradient(Test_params,Test_set)
    test_grads = test_grads[len(weights):]
    maxpos = np.argmax(np.abs(test_grads))

    max_op=pool_operators[maxpos]
    operator_circuits.append(max_op)
    weights=np.append(weights, 0.0)

    weights,_ = opt.step(cost_fn, weights, operator_circuits) # Step 11.
    E0 = cost_h_fn(g_stat,weights,operator_circuits)


    print(f"Epoch = {n+1}, G Energy = {E0:.8f} Ha, Sz = {Sz_fn(g_stat,weights,operator_circuits):.8f} Ha,")
    print("Number of gates = {}\n".format(len(operator_circuits)))
    configs=qml.specs(cost_h_fn)(g_stat,weights, excitations=operator_circuits)
    print("Circuit depth {}\n".format(configs['resources'].depth))

    if np.abs(E0-E0_pev)<cov:
        break

    E0_pev=E0


Epoch = 0, G Energy = -7.60898166 Ha
Number of gates = 0

Circuit depth 1

Epoch = 1, G Energy = -7.61248262 Ha, Sz = 0.00000000 Ha,
Number of gates = 1

Circuit depth 2

Epoch = 2, G Energy = -7.61715784 Ha, Sz = 0.00000000 Ha,
Number of gates = 2

Circuit depth 3

Epoch = 3, G Energy = -7.62030985 Ha, Sz = 0.00000000 Ha,
Number of gates = 3

Circuit depth 4

Epoch = 4, G Energy = -7.62145953 Ha, Sz = 0.00000000 Ha,
Number of gates = 4

Circuit depth 5

Epoch = 5, G Energy = -7.62183402 Ha, Sz = 0.00000000 Ha,
Number of gates = 5

Circuit depth 6

Epoch = 6, G Energy = -7.62221333 Ha, Sz = 0.00000000 Ha,
Number of gates = 6

Circuit depth 7

Epoch = 7, G Energy = -7.62268682 Ha, Sz = 0.00000000 Ha,
Number of gates = 7

Circuit depth 8

Epoch = 8, G Energy = -7.62325700 Ha, Sz = 0.00000000 Ha,
Number of gates = 8

Circuit depth 9

Epoch = 9, G Energy = -7.62387571 Ha, Sz = 0.00000000 Ha,
Number of gates = 9

Circuit depth 10

Epoch = 10, G Energy = -7.62446129 Ha, Sz = 0.00000000 Ha,
N

For $S_z=0$,

In [ ]:

e_trial_stat_sz0={'1001000000': 1,'0110000000': 1}
e_stat_sz0=bit_to_vec(e_trial_stat_sz0,num_qubits)

def circuit_h(state,params,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)

    return qml.expval(hamiltonian)


def circuit_fid(state,params,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)

    Re_w=weights[::-1]
    for i, excitation in enumerate(operator_circuits[::-1]):
        if len(excitation) == 4:
            qml.DoubleExcitation(-Re_w[i], wires=excitation)
        else:
            qml.SingleExcitation(-Re_w[i], wires=excitation)
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    return qml.probs(wires=range(num_qubits))

cost_h_fn = qml.QNode(circuit_h, dev, expansion_strategy="device")
fid_fn = qml.QNode(circuit_fid, dev)

def cost_fn(params, excitations):
    return cost_h_fn(e_stat_sz0,params, excitations)+5*fid_fn(e_stat_sz0,params, excitations)[0]

epochs=150

params = [0.0] * len(doubles)
params=np.array(params )

dev = qml.device("default.qubit", wires=num_qubits)

circuit_gradient = qml.grad(cost_fn, argnum=0)
grads_d = circuit_gradient(params, excitations=doubles)

params = [0.0] * len(singles)
params=np.array(params )

#dev = qml.device("default.qubit", wires=num_qubits)

#circuit_gradient = qml.grad(cost_fn, argnum=0)
grads_s = circuit_gradient(params, excitations=singles)

threshold=1e-5
doubles_select = [(doubles)[i] for i in range(len(doubles)) if abs(grads_d[i]) > threshold]
singles_select = [(singles)[i] for i in range(len(singles)) if abs(grads_s[i]) > threshold]

energy=[]
op_cir=doubles_select+singles_select
params = [0.0] * len(op_cir)
params=np.array(params )

opt = opt = qml.GradientDescentOptimizer(stepsize=0.03)

circuit_gradient = qml.grad(cost_fn, argnum=0)

E2 = cost_h_fn(e_stat_sz0,params, op_cir)


print(f"Epoch = 0, G Energy = {E2:.8f} Ha,")

print("Number of gates = {}\n".format(len(op_cir)))

configs=qml.specs(cost_h_fn)(g_stat,params, excitations=op_cir)

#print(configs)
print("Circuit depth {}\n".format(configs['resources'].depth))

conv=1e-7

E2_pev=0

for n in range(epochs):

    Test_set=op_cir+pool_operators
    Test_params=list(params)+[0.0]*len(pool_operators)
    Test_params=np.array(Test_params)

    test_grads = circuit_gradient(Test_params,Test_set)
    test_grads = test_grads[len(params):]
    maxpos = np.argmax(np.abs(test_grads))

    max_op=pool_operators[maxpos]
    op_cir.append(max_op)
    params=np.append(params, 0.0)

    params,_ = opt.step(cost_fn, params, op_cir) # Step 11.
    print('Cost function',cost_fn(params, op_cir))
    E2 = cost_h_fn(e_stat_sz0,params, op_cir)


    print(f"Epoch = {n+1}")
    print(f" The first excited state Energy = {E2:.8f} Ha, Sz= {Sz_fn(e_stat_sz0,params,op_cir):.8f}")
    print("Number of gates = {}\n".format(len(op_cir)))
    configs=qml.specs(cost_h_fn)(e_stat_sz0, params, excitations=op_cir)
    print("Circuit depth {}\n".format(configs['resources'].depth))

    if np.abs(E2-E2_pev)<cov:
        break
    E2_pev=E2


Epoch = 0, G Energy = -7.49074895 Ha,
Number of gates = 4

Circuit depth 3

Cost function -7.491147378838875
Epoch = 1
 The first excited state Energy = -7.49114742 Ha, Sz= 0.00000000
Number of gates = 5

Circuit depth 4

Cost function -7.491671058322511
Epoch = 2
 The first excited state Energy = -7.49167109 Ha, Sz= 0.00000000
Number of gates = 6

Circuit depth 4

Cost function -7.492313507917783
Epoch = 3
 The first excited state Energy = -7.49231356 Ha, Sz= -0.00000000
Number of gates = 7

Circuit depth 5

Cost function -7.49306564911569
Epoch = 4
 The first excited state Energy = -7.49306569 Ha, Sz= -0.00000000
Number of gates = 8

Circuit depth 5

Cost function -7.493918227433652
Epoch = 5
 The first excited state Energy = -7.49391824 Ha, Sz= -0.00000000
Number of gates = 9

Circuit depth 6

Cost function -7.494860678212293
Epoch = 6
 The first excited state Energy = -7.49486068 Ha, Sz= 0.00000000
Number of gates = 10

Circuit depth 6

Cost function -7.495881592050727
Epoch = 7
 T